In [1]:
import pandas as pd

url = r'https://drive.google.com/file/d/1EznuwPohSIzh0HZn5Fy0K5NApK3W4Clf/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
transaction = pd.read_parquet(url)

identity_url = 'https://drive.google.com/uc?id=1o0w9UnbUnf2mGk85KCYq6i2Ojv4UyZvR&export=download'
identity = pd.read_csv(identity_url)

df = pd.merge(transaction, identity, on='TransactionID', how='left')
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


## Train vs Test Splitting

In [2]:
import pandas as pd
#import sklearn as sklearn
from sklearn.model_selection import train_test_split

# load the data into a Pandas DataFrame

# define the feature and target variables
X = df.drop('isFraud', axis=1)
y = df.isFraud

# split the data into training and testing sets, stratifying on the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [3]:
y_train.value_counts()

0    455902
1     16530
Name: isFraud, dtype: int64

In [4]:
100*16530/(455902+16530)

3.498916246147594

## SMOTE

In [5]:
from imblearn.over_sampling import SMOTE

# create the SMOTE object
smote = SMOTE(sampling_strategy='auto')

In [7]:
# resample the data using SMOTE
#X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## AutoEncoders

In [22]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import minmax_scale



In [29]:
# Generate some random data
data = pd.read_csv('hotel_bookings_ohe.csv')

sData = minmax_scale(data, axis = 0) #scale data to auencoder

num_principleComponent = 10


In [30]:
# Define the input shape
input_shape = (sData.shape[1],)

# Define the encoding dimension
encoding_dim = num_principleComponent

# Define the input layer
input_data = Input(shape=input_shape)

# Define the encoder layers
encoded = Dense(20, activation='relu')(input_data)
encoded = Dense(10, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# Define the decoder layers
decoded = Dense(10, activation='relu')(encoded)
decoded = Dense(20, activation='relu')(decoded)
decoded = Dense(input_shape[0], activation='sigmoid')(decoded)

# Define the autoencoder model
autoencoder = Model(inputs=input_data, outputs=decoded)

# Define the encoder model
encoder = Model(inputs=input_data, outputs=encoded)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
decoder = Model(inputs=encoded_input, outputs=decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
autoencoder.fit(sData, sData, epochs=50, batch_size=32, shuffle=True)

# Use the encoder to get the reduced dimension representation of the data
encoded_data = pd.DataFrame(encoder.predict(sData))

# Print the shape of the reduced dimension representation
print(encoded_data.shape)


Epoch 1/50
3731/3731 [==============================] - 8s 2ms/step - loss: 0.0500
Epoch 2/50
3731/3731 [==============================] - 6s 2ms/step - loss: 0.0362
Epoch 3/50
3731/3731 [==============================] - 7s 2ms/step - loss: 0.0346
Epoch 4/50
3731/3731 [==============================] - 6s 2ms/step - loss: 0.0332
Epoch 5/50
3731/3731 [==============================] - 6s 2ms/step - loss: 0.0323
Epoch 6/50
3731/3731 [==============================] - 6s 2ms/step - loss: 0.0318
Epoch 7/50
3731/3731 [==============================] - 7s 2ms/step - loss: 0.0313
Epoch 8/50
3731/3731 [==============================] - 9s 3ms/step - loss: 0.0310
Epoch 9/50
3731/3731 [==============================] - 7s 2ms/step - loss: 0.0307
Epoch 10/50
3731/3731 [==============================] - 7s 2ms/step - loss: 0.0306
Epoch 11/50
3731/3731 [==============================] - 7s 2ms/step - loss: 0.0304
Epoch 12/50
3731/3731 [==============================] - 7s 2ms/step - loss: 0.0303
E

In [34]:
#the quality check of dimension reduction 
#A lower reconstruction error indicates a better quality of dimensionality reduction.

# Evaluate the autoencoder on the same data used for training to get the reconstruction error
loss = autoencoder.evaluate(sData, sData)
print("Reconstruction error:", loss)


3731/3731 [==============================] - 5s 1ms/step - loss: 0.0254
Reconstruction error: 0.025414563715457916


In [35]:
encoded_data

,0,1,2,3,4,5,6,7,8,9
0,5.531111,6.382500,0.0,22.040833,27.021603,0.332168,13.308737,0.0,1.350435,17.971134
1,5.527696,6.377851,0.0,22.041325,27.018457,0.330459,13.308756,0.0,1.356482,17.965590
2,8.571834,4.946552,0.0,22.139435,27.359840,2.159808,12.840506,0.0,0.000000,16.111317
3,6.955074,8.387858,0.0,17.979364,16.786715,2.942868,14.972704,0.0,0.000000,15.159703
4,10.047194,14.850857,0.0,14.638595,6.890122,6.756176,18.872902,0.0,0.000000,17.107143
...,...,...,...,...,...,...,...,...,...,...
119383,7.446973,14.507200,0.0,5.626014,11.456783,6.708698,12.522596,0.0,0.000000,8.866212
119384,8.495354,16.337532,0.0,4.575735,9.079448,7.622482,13.082062,0.0,0.000000,10.043538
119385,8.497374,16.349655,0.0,4.576653,9.128711,7.638931,13.098471,0.0,0.000000,10.011065
119386,8.485168,16.285006,0.0,4.572472,9.029105,7.580419,13.022150,0.0,0.000000,10.084586


In [36]:
0.007241828832775354

0.007241828832775354